In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
import numpy as np

In [2]:
# spark = SparkSession.builder.master('local[16]').getOrCreate()
sc = SparkContext('local[16]',)

spark = SparkSession(sc).builder.config('spark.driver.memory', '15g').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/30 12:47:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/30 12:47:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
schema = StructType() \
	.add('prot_access', StringType()) \
	.add('sequ_md5_dig', StringType()) \
	.add('seq_len', IntegerType()) \
	.add('analysis', StringType()) \
	.add('sign_access', StringType()) \
	.add('sign_descr', StringType()) \
	.add('start_loc', IntegerType()) \
	.add('stop_loc', IntegerType()) \
	.add('score', StringType()) \
	.add('status', StringType()) \
	.add('date', StringType()) \
	.add('iPro_access', StringType()) \
	.add('iPro_descr', StringType()) \
	.add('GO_annot', StringType()) \
	.add('pathway', StringType())



In [4]:
path = '/data/dataprocessing/interproscan/all_bacilli.tsv'

df = spark.read.csv(path, sep=r'\t', header=False, schema=schema)

If you don't want to use the `schema` then you can also use the following code to load in the data:

In [5]:
# df = spark.read.csv(path, sep=r'\t', header=False, inferSchema=True)

# col_names = ["protein_accession", "sequence_MD5", "sequence_length", "analysis", "signature_accession", "signature_description", "start_location",
# "stop_location", "score", "status", "date", "interpro_annot_accession", "interpro_annot_description", "GO", "pathway_annotations"]

# df = df.toDF(*col_names)

In [6]:
selection = df.select(['prot_access', 'seq_len', 'iPro_access', 'start_loc', 'stop_loc']).filter("iPro_access != '-'")

In [7]:
computed = selection.withColumn('difference', (col('stop_loc') - col('start_loc')) / col('seq_len')) \
	.select(['prot_access', 'iPro_access', 'difference'])

In [8]:
piv = computed.groupBy('prot_access').pivot('iPro_access').max()

In [9]:
cols = piv.columns[1:]

In [10]:
from pyspark.sql.functions import col, greatest, udf, array
from pyspark.sql.types import StringType
from pyspark.sql.functions import when

In [11]:
max_values = piv.withColumn('maxValue', greatest(*[col(x) for x in cols]))

In [12]:
col_arr = max_values.columns[1:]

In [13]:
def modify_values(r):
	for i in range(len(r[:-1])):
		if r[i] == r[-1] and r[i] > 0.9:
			return col_arr[i]

In [14]:
modify_values_udf = udf(modify_values, StringType())

In [15]:
max_df = max_values.withColumn('maxColumn', modify_values_udf(array(col_arr)))

In [16]:
cols = max_df.columns[1:-2]

In [17]:
t5 = max_df
for i in cols:
	t5 = t5.withColumn(i, when(col(i) > 0.0, 1).otherwise(0))

Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: Java heap space


Py4JJavaError: An error occurred while calling o19535.withColumn.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.trees.TreeNode.<init>(TreeNode.scala:233)
	at org.apache.spark.sql.catalyst.expressions.Expression.<init>(Expression.scala:86)
	at org.apache.spark.sql.catalyst.expressions.LeafExpression.<init>(Expression.scala:506)
	at org.apache.spark.sql.catalyst.analysis.GetColumnByOrdinal.<init>(unresolved.scala:576)
	at org.apache.spark.sql.catalyst.encoders.ExpressionEncoder$$anonfun$5.applyOrElse(ExpressionEncoder.scala:286)
	at org.apache.spark.sql.catalyst.encoders.ExpressionEncoder$$anonfun$5.applyOrElse(ExpressionEncoder.scala:281)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:584)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$Lambda$1279/0x0000000840a36440.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:176)
	at org.apache.spark.sql.c

In [22]:
sc.stop()

ConnectionRefusedError: [Errno 111] Connection refused